In [1]:
import pandas as pd
from datetime import timedelta
import os

In [2]:
# Load Data
df = pd.read_csv("C:\data\Retail.csv")

In [3]:
df.head()

,order_id,order_date,customer_name,city,state,product_name,category,quantity,price
0,1001,2024-01-24,Rohit,Hyderabad,Telangana,Shoes,Fashion,1,3000
1,1002,2024-02-25,Priya,Mumbai,Maharashtra,Laptop,Electronics,2,50000
2,1003,2024-04-07,Suman,Kolkata,West Bengal,Mouse,Electronics,3,800
3,1004,2024-02-24,Anjali,Jaipur,Rajasthan,Keyboard,Electronics,2,1500
4,1005,2024-03-09,Rahul,Bangalore,Karnataka,Mobile,Electronics,3,20000


In [4]:
# Calculate Revenue
df['Revenue'] = df['quantity'] * df['price']

In [5]:
df.head()

,order_id,order_date,customer_name,city,state,product_name,category,quantity,price,Revenue
0,1001,2024-01-24,Rohit,Hyderabad,Telangana,Shoes,Fashion,1,3000,3000
1,1002,2024-02-25,Priya,Mumbai,Maharashtra,Laptop,Electronics,2,50000,100000
2,1003,2024-04-07,Suman,Kolkata,West Bengal,Mouse,Electronics,3,800,2400
3,1004,2024-02-24,Anjali,Jaipur,Rajasthan,Keyboard,Electronics,2,1500,3000
4,1005,2024-03-09,Rahul,Bangalore,Karnataka,Mobile,Electronics,3,20000,60000


In [6]:
df.tail()

,order_id,order_date,customer_name,city,state,product_name,category,quantity,price,Revenue
195,1196,2024-04-18,Neha,Pune,Maharashtra,Keyboard,Electronics,1,1500,1500
196,1197,2024-03-14,Neha,Pune,Maharashtra,Mobile,Electronics,1,20000,20000
197,1198,2024-03-02,Rohit,Hyderabad,Telangana,Shoes,Fashion,3,3000,9000
198,1199,2024-02-22,Amit,Delhi,Delhi,Laptop,Electronics,1,50000,50000
199,1200,2024-03-29,Amit,Delhi,Delhi,Shoes,Fashion,1,3000,3000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       200 non-null    int64 
 1   order_date     200 non-null    object
 2   customer_name  200 non-null    object
 3   city           200 non-null    object
 4   state          200 non-null    object
 5   product_name   200 non-null    object
 6   category       200 non-null    object
 7   quantity       200 non-null    int64 
 8   price          200 non-null    int64 
 9   Revenue        200 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 15.8+ KB


In [8]:
# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       200 non-null    int64         
 1   order_date     200 non-null    datetime64[ns]
 2   customer_name  200 non-null    object        
 3   city           200 non-null    object        
 4   state          200 non-null    object        
 5   product_name   200 non-null    object        
 6   category       200 non-null    object        
 7   quantity       200 non-null    int64         
 8   price          200 non-null    int64         
 9   Revenue        200 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 15.8+ KB


In [11]:
df.describe()

,order_id,quantity,price,Revenue
count,200.000000,200.0000,200.00000,200.000000
mean,1100.500000,1.9600,10660.00000,20217.500000
std,57.879185,0.8378,16754.94077,34937.512686
min,1001.000000,1.0000,800.00000,800.000000
25%,1050.750000,1.0000,1500.00000,2500.000000
50%,1100.500000,2.0000,2800.00000,4500.000000
75%,1150.250000,3.0000,4000.00000,12000.000000
max,1200.000000,3.0000,50000.00000,150000.000000


In [12]:
# Set snapshot date (one day after last order)
snapshot_date = df['order_date'].max() + timedelta(days=1)

In [13]:
# Calculate Recency
recency = (
    df.groupby('customer_name')['order_date']
      .max()
      .reset_index()
)
recency['Recency'] = (snapshot_date - recency['order_date']).dt.days

In [14]:
recency = recency[['customer_name', 'Recency']]
print(recency)

  customer_name  Recency
0          Amit        6
1        Anjali        1
2         Kunal        2
3          Neha        1
4         Priya        4
5         Rahul        7
6         Rohit       12
7         Suman        1


In [15]:
# Calculate Frequency
frequency = (
    df.groupby('customer_name')['order_id']
      .nunique()
      .reset_index()
)

frequency.columns = ['customer_name', 'Frequency']
print(frequency)

  customer_name  Frequency
0          Amit         29
1        Anjali         18
2         Kunal         25
3          Neha         29
4         Priya         26
5         Rahul         22
6         Rohit         21
7         Suman         30


In [16]:
# Calculate Monetary
df['Monetary'] = df['quantity'] * df['price']

monetary = (
    df.groupby('customer_name')['Monetary']
      .sum()
      .reset_index()
)

print(monetary)

  customer_name  Monetary
0          Amit    735900
1        Anjali    380800
2         Kunal    670900
3          Neha    285200
4         Priya    487300
5         Rahul    395300
6         Rohit    413800
7         Suman    674300


In [17]:
# Merge Recency and Frequncy
rf = pd.merge(recency, frequency, on='customer_name', how='inner')
print(rf)

  customer_name  Recency  Frequency
0          Amit        6         29
1        Anjali        1         18
2         Kunal        2         25
3          Neha        1         29
4         Priya        4         26
5         Rahul        7         22
6         Rohit       12         21
7         Suman        1         30


In [18]:
# Merge RF and Monetary
rfm = pd.merge(rf, monetary, on='customer_name', how='inner')
print(rfm)

  customer_name  Recency  Frequency  Monetary
0          Amit        6         29    735900
1        Anjali        1         18    380800
2         Kunal        2         25    670900
3          Neha        1         29    285200
4         Priya        4         26    487300
5         Rahul        7         22    395300
6         Rohit       12         21    413800
7         Suman        1         30    674300


In [19]:
# Customer Segmentation
def segment_customer(row):
    if row['Recency'] <= 2 and row['Frequency'] >= 25 and row['Monetary'] >= 600000:
        return 'Champion'
    elif row['Frequency'] >= 20 and row['Monetary'] >= 300000:
        return 'Loyalist'
    else:
        return 'Hibernating'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)
rfm

,customer_name,Recency,Frequency,Monetary,Segment
0,Amit,6,29,735900,Loyalist
1,Anjali,1,18,380800,Hibernating
2,Kunal,2,25,670900,Champion
3,Neha,1,29,285200,Hibernating
4,Priya,4,26,487300,Loyalist
5,Rahul,7,22,395300,Loyalist
6,Rohit,12,21,413800,Loyalist
7,Suman,1,30,674300,Champion


In [20]:
rfm['Segment'].value_counts()

Loyalist       4
Hibernating    2
Champion       2
Name: Segment, dtype: int64

In [21]:
# Export Outputs
rfm.to_csv("rfm_output_customer360.csv", index=False)

In [22]:
os.system("python rfm_analysis.py")

1